In [1]:
# Тюнинг
import optuna as opt
from lightgbm import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import BaggingClassifier, StackingClassifier
from sklearn.model_selection import cross_val_score, train_test_split, StratifiedKFold
from sklearn.metrics import f1_score

# Пайплайн
from sklearn.pipeline import Pipeline
from sklearn.base import TransformerMixin, BaseEstimator

# Данные
import os
import pandas as pd
import numpy as np
from category_encoders import BinaryEncoder, OneHotEncoder
from sklearn.preprocessing import RobustScaler

In [2]:
# Пути
ROOT = os.getcwd()
TRAIN_DATASET = os.path.join(ROOT, '../data/train_AIC.csv')
BALANCED_DATASET = os.path.join(ROOT, '../data/balanced_train.csv')
TEST_DATASET = os.path.join(ROOT, '../data/test_AIC.csv')
SUBMISSION_PATH = os.path.join(ROOT, '../submissions/')

def save_submission(model, subname):
    subname = os.path.join(SUBMISSION_PATH, f'{subname}.csv')
    preds = model.predict(test_df)
    submit_df = pd.DataFrame({'id': test_df.index, 'value': preds})
    submit_df.to_csv(subname, index=False)

# Загрузка
train_df = pd.read_csv(TRAIN_DATASET)
# balanced_df = pd.read_csv(BALANCED_DATASET, index_col=0)
# train_df = balanced_df.copy()
test_df = pd.read_csv(TEST_DATASET)

# first_negatives = train_df[train_df['y'] == 0][:train_df[train_df['y'] == 1]['y'].count()]
# train_df = pd.concat([train_df[train_df['y'] == 1], first_negatives])

# def random_undersample(df):
#     neg_count, pos_count = np.bincount(df['y'])
#     pos_df = df[df['y'] == 1]
#     neg_df = df[df['y'] == 0]
#     neg_df = neg_df.sample(n=pos_count, random_state=1708)
#     return pd.concat([pos_df, neg_df])

# balanced_df = random_undersample(train_df)

# Нормализация
train_df = train_df[train_df['Длительность'] < 500]
train_df = train_df[(train_df['Сумма'] > 2) & (train_df['Сумма'] < 10)]
train_df = train_df[train_df['До поставки'] < 300]

X, y = train_df.iloc[:, :-1], train_df.iloc[:, -1]   
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)

NameError: name 'pd' is not defined

In [24]:
# Категориальные фичи
cat_features = [
    'Provider', 'Purchasing Organization', 'Company Code',
    'Provider Purchaser', 'Factory Material'
    ]

# Фичи на масштабирование
scale_features = [
    'Position Count', 'Duration', 'ETC Delivery',
    'Changes After Approvals', 'Order Approval 1', 'Order Approval 2',
    'Order Approval 3', 'Sum', 'Delivery Date'
]

# Фичи на дроп
drop_features = [
    'Change on Paper', 'Amount', 'Category Manager', 'Factory', 
    'Material', 'Cancel Complete Release', 'Approval Cycles',
    'Change Delivery Date 15', 'Change Delivery Date 30', 
    'EI', 'Delivery Option', 'Days Between 4_5',
    'Days Between 5_6', 'Days Between 6_7',
    'Material Group', 'Days Between 1_2', 'Days Between 2_3', 
    'Days Between 3_4', 'Days Between 7_8', 'NRP', 'Operations Manager', 
    'Month1', 'Month2', 'Month3', 'Weekday'
    ]

# Новые имена фич
rename_cols = [
    'Provider', 'Material', 'Category Manager', 'Operations Manager',
    'Factory', 'Purchasing Organization', 'Purchasing Group', 
    'Company Code', 'EI', 'Material Group', 'Delivery Option', 'NRP',
    'Duration', 'ETC Delivery', 'Month1', 'Month2', 'Month3', 'Weekday',
    'Sum', 'Position Count', 'Amount', 'Handlers 7', 'Handlers 15', 
    'Handlers 30', 'Order Approval 1', 'Order Approval 2', 'Order Approval 3',
    'Change Delivery Date 7', 'Change Delivery Date 15', 'Change Delivery Date 30',
    'Cancel Complete Release', 'Change on Paper', 'Delivery Date', 
    'Approval Cycles', 'Changes After Approvals', 'Days Between 0_1', 
    'Days Between 1_2', 'Days Between 2_3', 'Days Between 3_4', 'Days Between 4_5', 
    'Days Between 5_6', 'Days Between 6_7', 'Days Between 7_8'
    ]

In [25]:
# Препроцессоры
class DataPreprocessor(BaseEstimator, TransformerMixin):
    """ Предобработчик данных """
    def __init__(self, cat_features, scale_features,
                 drop_features, rename_cols, transform_train=True):
        self.transform_train = transform_train
        self.cat_features = cat_features

        self.bin_encoder = BinaryEncoder(cols=cat_features)
        self.robust_scaler = RobustScaler()
        # self.spline = SplineTransformer(n_knots=3, degree=8, extrapolation='linear')

        self.rename_cols = rename_cols

        self.drop_features = drop_features
        self.scale_features = scale_features

    def fit(self, X, y=None):
        # Создаём копию датасета
        X_ = X.copy()
        X_.columns = self.rename_cols

        # Временные фичи
        X_['Weekday'] += 1
        X_['day_sin'] = np.sin(np.pi * 2 * X_['Weekday'] / 7)
        X_['day_cos'] = np.cos(np.pi * 2 * X_['Weekday'] / 7)
        X_['month1_sin'] = np.sin(np.pi * 2 * X_['Month1'] / 12)
        X_['month1_cos'] = np.cos(np.pi * 2 * X_['Month1'] / 12)
        X_['month2_sin'] = np.sin(np.pi * 2 * X_['Month2'] / 12)
        X_['month2_cos'] = np.cos(np.pi * 2 * X_['Month2'] / 12)
        X_['month3_sin'] = np.sin(np.pi * 2 * X_['Month3'] / 12)
        X_['month3_cos'] = np.cos(np.pi * 2 * X_['Month3'] / 12)

        # Экстракция фич
        X_['Provider Purchaser'] = [f'{x}_{y}' for x, y in zip(X_['Provider'].values, X_['Purchasing Organization'].values)]
        X_['Factory Material'] = [f'{x}_{y}' for x, y in zip(X_['Factory'].values, X_['Material'].values)]

        # Нормализация
        self.robust_scaler.fit(X_[self.scale_features])

        # Категориальные фичи        
        X_ = self.bin_encoder.fit_transform(X_)

        X_ = X_.drop(self.drop_features, axis=1)
        
        return self
    
    def transform(self, X):
        # Создаём копию датасета
        X_ = X.copy()
        X_.columns = self.rename_cols

        # Временные фичи
        X_['Weekday'] += 1
        X_['day_sin'] = np.sin(np.pi * 2 * X_['Weekday'] / 7)
        X_['day_cos'] = np.cos(np.pi * 2 * X_['Weekday'] / 7)
        X_['month1_sin'] = np.sin(np.pi * 2 * X_['Month1'] / 12)
        X_['month1_cos'] = np.cos(np.pi * 2 * X_['Month1'] / 12)
        X_['month2_sin'] = np.sin(np.pi * 2 * X_['Month2'] / 12)
        X_['month2_cos'] = np.cos(np.pi * 2 * X_['Month2'] / 12)
        X_['month3_sin'] = np.sin(np.pi * 2 * X_['Month3'] / 12)
        X_['month3_cos'] = np.cos(np.pi * 2 * X_['Month3'] / 12)
        
        # Нормализация
        X_[self.scale_features] = self.robust_scaler.transform(X_[self.scale_features])

        # Экстракция фич
        X_['Provider Purchaser'] = [f'{x}_{y}' for x, y in zip(X_['Provider'].values, X_['Purchasing Organization'].values)]
        X_['Factory Material'] = [f'{x}_{y}' for x, y in zip(X_['Factory'].values, X_['Material'].values)]

        # Категориальные фичи
        X_ = self.bin_encoder.transform(X_)

        X_ = X_.drop(self.drop_features, axis=1)

        return X_

In [21]:
# Функция оптимизации
def objective(trial: opt.Trial):
    # Параметры
    n_estimators = trial.suggest_int('n_estimators', 100, 1000, 50)
    max_depth = trial.suggest_int('max_depth', 6, 16)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 128)
    min_samples_split = trial.suggest_int('min_samples', 2, 32)
   
    # Модель
    data_preprocessor = DataPreprocessor(cat_features, scale_features, drop_features, rename_cols)
    model = RandomForestClassifier(
        criterion='log_loss',
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_leaf=min_samples_leaf,
        min_samples_split=min_samples_split,
        class_weight='balanced',
        n_jobs=-1
    )
    
    pipeline = Pipeline([
        ('data_preproc', data_preprocessor),
        ('model', model)
    ])
    
    cv_score = cross_val_score(pipeline, X_train, y_train, cv=StratifiedKFold(n_splits=5), scoring='f1_macro', n_jobs=-1)
    accuracy = cv_score.mean()

    return accuracy

In [ ]:
study = opt.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

In [15]:
params_list = [{
    'learning_rate': 0.3, 
    'n_estimators': 1000, 
    'max_depth': 8, 
    'max_bin': 128, 
    'num_leaves': 128, 
    'reg_lambda': 0.2,
    }, {
    'learning_rate': 0.1, 
    'n_estimators': 1500, 
    'max_depth': 16, 
    'max_bin': 128, 
    'num_leaves': 128, 
    'reg_lambda': 0.2,
    }, {
    'learning_rate': 0.6, 
    'n_estimators': 1000, 
    'max_depth': 6, 
    'max_bin': 256, 
    'num_leaves': 128, 
    'reg_lambda': 0.5,
    }, {
    'learning_rate': 0.15, 
    'n_estimators': 1500, 
    'max_depth': 12, 
    'max_bin': 128, 
    'num_leaves': 128, 
    'reg_lambda': 0.4,
    }]

rfk_params = {'n_estimators': 200, 'max_depth': 12, 'min_samples_leaf': 41, 'min_samples': 17}

# Модель
data_preprocessor = DataPreprocessor(cat_features, scale_features, drop_features, rename_cols)

estimators = []
for index, params in enumerate(params_list):
    estimator = LGBMClassifier(
        **params,
        n_jobs=-1,
        force_col_wise=True,
        is_unbalance=True
    )
    packed = (f'model_{index}', estimator)
    estimators.append(packed)

model = StackingClassifier(estimators=estimators, n_jobs=-1)

pipeline = Pipeline([
    ('data_preproc', data_preprocessor),
    ('model', model)
])

In [16]:
pipeline.fit(X_train, y_train)
f1_score(y_test, pipeline.predict(X_test), average='macro')

0.9917429698917423

In [15]:
pipeline.fit(X_train, y_train)
f1_score(y_test, pipeline.predict(X_test), average='macro')

0.9048908613978871

In [17]:
save_submission(pipeline, 'submission_LGBM_ENS')